### 데이터 수집/획득 - Level 2

#### 사용기술
- open api 필요 (Get)
  - 관련사이트
    - dev.naver.com
    - dev.kakao.com
  - Client_ID (관리)
    - pP7lQhkZAx70dyk3sLcJ
  - Client Secret (관리)
    - twyoc8OdDG
  - API 문서
    - https://developers.naver.com/docs/search/news/
  - 검색 URL (응답 데이터 json)
    - https://openapi.naver.com/v1/search/news.json

  - 예시

```
curl "https://openapi.naver.com/v1/search/news.xml?query=%EC%A3%BC%EC%8B%9D&display=10&start=1&sort=sim" \
    -H "X-Naver-Client-Id: {애플리케이션 등록 시 발급받은 client id 값}" \
    -H "X-Naver-Client-Secret: {애플리케이션 등록 시 발급받은 client secret 값}" -v
```


- request 모듈 필요. => 통신 수행(http)
  - get방식을 주로 사용
  - 개인별 인증키는 헤더에 숨겨서 전송.

#### 구현

In [19]:
# 1. 필요한 모듈 가져오기.
import urllib.request
import os
import sys

In [7]:
# 2. 환경변수, 통신에 필요한 Key를 정의.
Client_ID     = '_HV4Mav6gAsfgons6mTA'
Client_Secret = 'esqEqzOGLv'

In [8]:
# 3. URL 정의.
naver_news_url= 'https://openapi.naver.com/v1/search/news.json'
naver_news_url

'https://openapi.naver.com/v1/search/news.json'

In [10]:
# 4. 파라미터 정의.
# 한글이 검색어인 경우, utf-8 인코딩 처리가 필요가 필요. 
# 한글 => %EA%AA..
keyword = urllib.parse.quote('블랙핑크')
keyword
param   = 'query=' + keyword
param

'query=%EB%B8%94%EB%9E%99%ED%95%91%ED%81%AC'

In [11]:
# 5. 최종 get방식으로 요청하는 url. 
req_url = f'{naver_news_url}?{param}'
req_url

'https://openapi.naver.com/v1/search/news.json?query=%EB%B8%94%EB%9E%99%ED%95%91%ED%81%AC'

In [12]:
# 6. 통신 객체 생성.
# from urllib.request import Request
request = urllib.request.Request( req_url )

In [13]:
# 7. 클라이언트 키, 보안 키 헤더에 등록.
'''
-H "X-Naver-Client-Id: {애플리케이션 등록 시 발급받은 client id 값}"
-H "X-Naver-Client-Secret: {애플리케이션 등록 시 발급받은 client secret 값}" -v
'''
request.add_header( 'X-Naver-Client-Id',     Client_ID     )
request.add_header( 'X-Naver-Client-Secret', Client_Secret )

In [14]:
# 8. 통신.
response = urllib.request.urlopen( request )
response

In [15]:
# 9. 응답코드 확인 -> 200 : http로 통신하고나서 오는 응답코드중 정상을 의미.
# 401 : 권한오류, 인증오류
# 404 : 해당 페이지가 없음.
# 405 : 해당 권한이 없음.
# 500 : 서버 내부 에러.
response.getcode()

200

In [16]:
# 10. 응답 데이터를 json 객체로 로드. => 딕녀서리, 리스트로 조합 구성.
import json

# 응답 코드가 정상적이지 않을 경우 오류 상황을 반환.
if response.getcode() == 200:
    res = json.load( response )
else:
    print('error', response.getcode())

In [18]:
# 11. 파싱. 
# 응답된 데이터에서 description 항목을 모두 출력.
print(len( res['items'] ))

for news in res['items'] :
  #print( news['description'] )

  # 데이터들을 좀 더 가공해서 데이터베이스에 적재할 필요가 있어 보임. <- 전처리(정규식, 기타..)
    print( news['title'].replace('<b>','').replace('</b>','') )

10
로제, 싸이 기록 경신…한류 이끄는 女솔로 잇단 컴백
&quot;주얼리만 6870만원&quot;..블랙핑크 리사, 명품이 픽한 아이돌 1위 ('TMI NEWS')[...
블랙핑크 아디다스 광고에 네티즌 탄식한 이유
'지디♥' 제니, 아찔한 레드 룩…한복 아니고 &quot;여기 제품&quot;
블랙핑크 로제, 美 '지미 팰런쇼' 특급 무대… MV 1억뷰 눈앞
블랙핑크 로제, 美 '지미 팰런쇼' 출연…On The Ground 미국 첫 무대 펼쳐
블랙핑크 지수, 깜찍한 토끼 포즈로 인증샷 “블링크 사랑해요”[M+★SNS]
블랙핑크 로제, 발매 당일 28만장 판매…주문량 50만장 돌파 '女솔로 최다'
미리 만나는 2021 가을/겨울 컬렉션 #프라다 #Prada
블랙핑크 로제 역시 ‘빛이 나는 솔로’ | 신곡 ‘온 더 그라운드’ 유튜브·...


#### 데이터 적제
- 파일 (반정형)
  - csv, xls(x)
  - json
- 데이터 베이스(스키마가 외부에 존재)
  - RDBMS
    - 기업형 디비, Ms-sql, My-sql, Oracle(비공유 디스크),..    
    - 현재 작업 기준에서는 코랩에서 작동시, **고정 IP 나 도메인을 가진 데이터베이스**를 활용하여 저장해야함.
      - aws 사용.
  - No-SQL : 비관계형 데이터 베이스 기반 sql.
    - 몽고디비 => 로그 저장.
    
- 절차
  - 데이터 구조 : [ {},{},{},... ] 준비
  - pandas를 이용하여 DataFrame 구성
  - pymysql + sqlalchemy 이용하여 접속
  - 데이터를 데이터베이스에 적제
  - 연결 종료
  
- 데이터 베이스 연결에 필요한 모듈 설치
    - !pip install pymysql

In [38]:
# 1. 필요 모듈 가져오기.
import pandas as pd
from sqlalchemy import create_engine
import pandas.io.sql as pSql
import pymysql

In [27]:
# 2. 수집한 데이터 프레임화.
df = pd.DataFrame.from_dict( res['items'] )
df.head()

,title,originallink,link,description,pubDate
0,"로제, 싸이 기록 경신…한류 이끄는 女솔로 잇단 컴백",http://news.tvchosun.com/site/data/html_dir/20...,https://news.naver.com/main/read.nhn?mode=LSD&...,[리포트] &quot;내가 필요한 모든 것은 여기 이 땅에 있어&quot; 미국의 ...,"Wed, 17 Mar 2021 21:47:00 +0900"
1,"&quot;주얼리만 6870만원&quot;..<b>블랙핑크</b> 리사, 명품이 픽...",http://www.osen.co.kr/article/G1111540882,https://news.naver.com/main/read.nhn?mode=LSD&...,'TMI NEWS' 1위는 셀린X 앰배서더로 활약 중인 <b>블랙핑크</b> 리사....,"Wed, 17 Mar 2021 21:27:00 +0900"
2,<b>블랙핑크</b> 아디다스 광고에 네티즌 탄식한 이유,http://star.mt.co.kr/stview.php?no=20210317210...,https://news.naver.com/main/read.nhn?mode=LSD&...,"&quot;아, 저 세상 다리&quot; 걸그룹 <b>블랙핑크</b>(BLACKPI...","Wed, 17 Mar 2021 21:15:00 +0900"
3,"'지디♥' 제니, 아찔한 레드 룩…한복 아니고 &quot;여기 제품&quot;",http://news.mt.co.kr/mtview.php?no=20210317095...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 <b>블랙핑크</b> 제니가 매혹적인 블랙 앤 레드 룩을 선보였다. 지난 16...,"Wed, 17 Mar 2021 21:01:00 +0900"
4,"<b>블랙핑크</b> 로제, 美 '지미 팰런쇼' 특급 무대… MV 1억뷰 눈앞",https://sports.hankooki.com/lpage/entv/202103/...,https://sports.hankooki.com/lpage/entv/202103/...,<b>블랙핑크</b> 로제가 세계 최대 팝 시장인 미국에서의 솔로 데뷔 무대를 성공...,"Wed, 17 Mar 2021 20:31:00 +0900"


In [28]:
# 3. 데이터 베이스 연결 문자열 준비.
id        = 'root'
pw        = '##비공개##'
domain    = '######.rds.amazonaws.com'
port      = 3306
database  = 'python_db'

db_url = f'mysql+pymysql://{id}:{pw}@{domain}:{port}/{database}'
engine = create_engine( db_url, encoding='utf8' )

# 데이터 베이스 연결.
conn   = engine.connect()

# 데이터 입력.
df.to_sql( name='tbl_news', con=conn, if_exists='append', index=False )

# 데이터 베이스 해제.
conn.close()